In [ ]:
#Task 1 (Проаналізувати кореляцію характеристики магазину та населення)

In [ ]:
import pandas as pd

In [ ]:
pop_df = pd.read_excel('Population.xls')

In [ ]:
store_df = pd.read_excel('Stores.xls')

In [ ]:
pop_df['lat'] = pop_df['lat'].round(6)
pop_df['lon'] = pop_df['lon'].round(6)

In [ ]:
store_df['lat'] = store_df['lat'].round(6)
store_df['lon'] = store_df['lon'].round(6)

In [ ]:
pop_df

In [ ]:
store_df['id'] = store_df.reset_index().index + 1
store_df

In [ ]:
from scipy.spatial import cKDTree

In [ ]:
tree = cKDTree(store_df[['lat', 'lon']].values)

In [ ]:
distances, indices = tree.query(pop_df[['lat', 'lon']].values)

In [ ]:
pop_df['id'] = store_df.iloc[indices]['id'].values

In [ ]:
sorted_pop_df = pop_df.sort_values(by='id')
sorted_pop_df

In [ ]:
mean_pop = sorted_pop_df.groupby('id').mean()
mean_pop

In [ ]:
merged_df = store_df.merge(sorted_pop_df, on='id')
merged_df

In [ ]:
merged_dfm = store_df.merge(mean_pop, on='id')
merged_dfm = merged_dfm.rename(columns={'metric population': 'metric_populationm'})
merged_dfm = merged_dfm.rename(columns={'Metric Store': 'Metric_Store'})
merged_dfm = merged_dfm.drop(['lon_y', 'lat_y'], axis=1)
merged_dfm

In [ ]:
merged_dfm['cor']=merged_dfm.metric_populationm/merged_dfm.Metric_Store 
merged_dfm



In [ ]:
min_cor = merged_dfm['cor'].min()
min_cor

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(50, 40))

plt.bar(merged_dfm['Store'], merged_dfm['cor'])
plt.xticks(rotation=45, ha='right', fontsize=23) 
plt.title('Кореляція Населення з Metric_Store', fontsize=56)
plt.xlabel('Магазин', fontsize=40)
plt.ylabel('Кореляція', fontsize=40)
plt.yticks(fontsize=20)
plt.axhline(y=1, color='r', linestyle='-', linewidth=6)
plt.show()

In [ ]:
#Task 3 (Запропонувати найкращі локації для відкриття нового store (за допомоги аналітичних досліджень або моделі))

In [ ]:
selected_stores = merged_dfm[merged_dfm['cor'] < 0.78]
selected_stores

In [ ]:
pip install geopy

In [ ]:
filtered_df = sorted_pop_df.loc[sorted_pop_df['id'].isin(id_list)].sort_values('id')
filtered_df

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def distance(lat1, lon1, lat2, lon2):
    R = 6371  # радиус Земли в км
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [ ]:
from geopy.distance import geodesic

# создаем словарь с координатами магазинов
stores_coords = {row['id']: (row['lat_x'], row['lon_x']) for idx, row in selected_stores.iterrows()}

# функция для расчета расстояния между координатами
def calculate_distance(row):
    store_coords = stores_coords.get(row['id'], None)
    if store_coords is None:
        return None
    return geodesic((row['lat'], row['lon']), store_coords).km

# создаем новый столбец с расстоянием
filtered_df['distance'] = filtered_df.apply(calculate_distance, axis=1)



In [ ]:
filtered_df

In [ ]:
filtered_df['id'].value_counts()

In [ ]:
import numpy as np

In [ ]:
new_stores


In [ ]:
# Розбиваю датафрейм на групи по id
grouped = filtered_df.groupby('id')

# Рахую середню дистанцію и средню metric_population для кожної группы id
grouped_means = grouped.mean()

# Для кожної групи id находжу координати, які відповідають мінімальній середній дистанцїї до магазину та середній metric_population
optimal_coords = []
for group_id, group_data in grouped:
    min_distance_mean = group_data['distance'].mean()
    min_pop_mean = group_data['metric population'].mean()
    min_distance_idx = group_data['distance'].idxmin()
    min_pop_idx = group_data['metric population'].idxmax()
    optimal_lat = group_data.loc[[min_distance_idx, min_pop_idx], 'lat'].mean()
    optimal_lon = group_data.loc[[min_distance_idx, min_pop_idx], 'lon'].mean()
    optimal_coords.append({'id': group_id, 'lat': optimal_lat, 'lon': optimal_lon})


optimal_df = pd.DataFrame(optimal_coords)


print(optimal_df)


In [ ]:
#id з номерами 34, 44 та 62 не розглядаємо через малу кількість даних щодо metric population
номер 77 співпадає вже з наявним магазином, тому його не розглядаємо.
А ось 59, 61, 87 вважаю перспективними координатами.

